In [73]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [74]:
import numpy as np
import pyarrow as pa
import pyspark as ps
import random
import re

In [75]:
sp = [spark.read.load('hdfs://orion11:14001/cleaned_books/{:d}'.format(i),
                     format='csv',
                     sep=',') for i in range(10)]
sp[0].printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [77]:
# for each book: absolute length of novel, average word length, and count of distinct words used
# rdds are ( title, author, text ) where text is a literal block of text with commas

In [78]:
# ( title, author, text )
# ( title, abs_len, [ text ] )
# ( title, abs_len, num_words, sum_word_lengths, [ text ] )
# ( title, abs_len, num_words, sum_word_lengths, [ text ] )
# ( title, ( abs_len, avg_word_length, num_distinct ) )

srdds = []
for i in range(10):
    srd = sp[i].rdd \
        .map(lambda x: ( x[0], len(x[0])+len(x[1])+len(x[2]), x[2].split(',') )) \
        .map(lambda x: ( x[0], x[1], len(x[2]), sum([len(w) for w in x[2]]), x[2] )) \
        .map(lambda x: ( x[0], ( x[1], x[3]/x[2], len(set(x[4])) ) ))
    srdds.append(srd)

srdd = srdds[0]
for i in range(1,10):
    srdd.join(srdds[i])

In [79]:
scales = srdd \
    .map(lambda x: np.vstack(x[1]) ) \
    .reduce(lambda a,b: np.hstack((a,b)))

avs_stds = np.hstack(( np.vstack(np.mean(scales,axis=1)), np.vstack(np.std(scales,axis=1)) ))
scales -= avs_stds[:,:1]
scales = np.divide(scales, avs_stds[:,1:], out=np.zeros_like(scales), where=avs_stds[:,1:]!=0).T

In [80]:
score_sums = np.sum(scales, axis=1)
sophistry = np.hstack((np.vstack(srdd.map(lambda x: [x[0]]).collect()), scales, np.vstack(score_sums)))
reorder = np.argsort(score_sums)[::-1]

remarks = ['. Verily', '. Quite', '. Indubitably', ', ole chap']
ends = ['.','!']

i = 0
for s in sophistry[reorder]:
    print("\n'{}' is {} measures sophisticated{}{}".format(s[0], s[-1][:4], np.random.choice(remarks), np.random.choice(ends)))
    if i >= 19:
        break
    i+=1


'The Golden Bough' is 8.82 measures sophisticated. Quite.

'Andivius Hedulio' is 4.57 measures sophisticated, ole chap.

'The Last Chronicle of Barset' is 4.26 measures sophisticated. Verily.

'South Wind' is 4.07 measures sophisticated, ole chap!

'Medical Essays' is 4.00 measures sophisticated. Verily!

'Jean-Christophe, Vol. I' is 3.99 measures sophisticated. Verily.

'The Memoirs of General W. T. Sherman, v1' is 3.54 measures sophisticated. Quite!

'Now It Can Be Told' is 3.37 measures sophisticated. Quite!

'The French in the Heart of America' is 3.18 measures sophisticated. Verily!

'A Tramp Abroad' is 3.17 measures sophisticated. Indubitably.

'Amelia (Complete)' is 2.90 measures sophisticated. Verily.

'Janice Meredith' is 2.71 measures sophisticated. Verily.

'The Principles Of Aesthetics' is 2.71 measures sophisticated, ole chap.

'Zenobia' is 2.56 measures sophisticated. Verily.

'The History of England from the Accession of James II, Vol. 5' is 2.55 measures sophisticated.

In [81]:
print(timeit.default_timer()-start)

5.3900826489552855
